<a href="https://colab.research.google.com/github/vedupandya/NLP/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datasets

In [2]:
dataset = datasets.load_dataset("cnn_dailymail",'1.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
data = dataset['train']['article'][:100]

In [4]:
from collections import Counter
from itertools import combinations
import numpy as np
import re
import gensim
import spacy

In [5]:
nlp = spacy.load("en_core_web_sm")
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [6]:
noun_phrases = []
for i in range(100):
    text = data[i]
    doc = nlp(text)
    for chunk in doc.noun_chunks:
        filtered_phrase = ' '.join(token.text for token in chunk if token.is_alpha and token.text.lower() not in stopwords)
        if filtered_phrase:
            noun_phrases.append(filtered_phrase.lower())

## Word2Vec

In [7]:
tokenized_phrases = [phrase.split() for phrase in noun_phrases]

In [ ]:
# Train Word2Vec model
model = gensim.models.Word2Vec(tokenized_phrases, min_count=1, vector_size=100,window=5,workers=-1)

# Top - k

In [15]:
k = 10
all_words = noun_phrases
word_counts = Counter(all_words)
top_phrases = word_counts.most_common(k)

print(f"Top {k} most frequent phrases:")
for phrase, count in top_phrases:
    print(f"{phrase}: {count}")

Top 10 most frequent phrases:
friend: 106
cnn: 105
mail: 101
e: 99
people: 97
iraq: 61
time: 49
children: 43
police: 40
president: 38


In [16]:
words = [top_phrases[i][0] for i in range(k)]
for i in words:
  similar_words = model.wv.most_similar(i, topn=10)
  print(f"Words similar to '{i}'")
  for word, similarity in similar_words:
    print(word)
  print("-"*50)

Words similar to 'friend'
registration
law
benz
gunshot
hero
hashimi
brake
clinic
n
gel
--------------------------------------------------
Words similar to 'cnn'
days
investment
pvt
osasuna
kivu
token
raphael
follow
press
necessities
--------------------------------------------------
Words similar to 'mail'
cancellation
analysis
offers
elton
dispute
allies
hurricanes
district
ecology
raspberry
--------------------------------------------------
Words similar to 'e'
seizures
algiers
borussia
knocked
whacked
threats
tissues
brazilians
raleigh
oncologist
--------------------------------------------------
Words similar to 'people'
supporter
carmindy
kh
bookstores
smoking
permanent
roma
upcoming
glass
haas
--------------------------------------------------
Words similar to 'iraq'
department
waco
diffusion
professional
illegal
officials
ill
turns
ipcc
wonderful
--------------------------------------------------
Words similar to 'time'
alternative
rescue
tennessee
blackface
partner
oil
walsh
o

### Extra

In [10]:
# Function to compute the phrase vectors
def get_phrase_vector(phrase):
    phrase_vector = np.zeros(model.vector_size)
    for word in phrase:
        if word in model.wv:
            phrase_vector += model.wv[word]
    return phrase_vector

In [11]:
pvec = [get_phrase_vector(p.split()) for p in noun_phrases]

In [18]:
noun_phrases[5]

'reported million million fortune'

In [12]:
result = model.wv.similar_by_vector(pvec[5], topn=5)
result

[('million', 0.8218741416931152),
 ('reported', 0.41035813093185425),
 ('fortune', 0.32655957341194153),
 ('massa', 0.32347962260246277),
 ('dress', 0.31504026055336)]